In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

from tensorflow.keras.datasets import mnist


# Constants and datasets

In [8]:
z_dim = 100

# Gans are very sensitive to hyperparameters, choosing right parameters for
# the optimizer can make the difference between learning and not learning.
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0005, beta_1=0.6)

# Having bigger batch improves gradient stima
batch_size = 128

Few notable things:

- we only need the X_train data
- the labels are scaled to stay between -1 and 1 (using an hyperbolic tangent with the output layer seems to give better results)

In [9]:
(X_train, _), (_, _) = mnist.load_data()

# usually we normalized with 255 but now we stay between 
X_train = X_train / 127.5 - 1.
X_train = X_train.reshape(-1,28,28,1)

# Generator definition

In [10]:
# DC Gan Idea
G_in = Input(shape=[z_dim])
x = Dense(1000, activation="relu")(G_in)
x = BatchNormalization()(x)
x = Dense(512, activation="relu")(x)
x = BatchNormalization()(x)
x = Dense(784, activation="tanh")(x)
G_out = Reshape([28,28,1])(x)
G = Model(G_in, G_out)

# Discriminator definition

In [11]:
D_in = Input(shape=[28,28,1])
x = Flatten()(D_in)
x = Dense(256, activation="relu")(x)
x = Dropout(rate=0.25)(x)
x = Dense(128, activation="sigmoid")(x)
x = Dropout(rate=0.25)(x)
D_out = Dense(1, activation="sigmoid")(x)

D = Model(D_in, D_out)

# Let's keep under control few important metrics (tp and tn in particular)
D.compile(loss="binary_crossentropy", optimizer=optimizer, 
    metrics=["accuracy",tf.keras.metrics.TruePositives(),tf.keras.metrics.TrueNegatives()])

# GAN definition

In [12]:
gan_in = Input(shape=[z_dim])
gan_out = D(G(gan_in))
gan = Model(gan_in, gan_out)

# we obtain -1/2 E_z log D(G(z)) simply by flipping the labels
# b_cross_entropy(y,k) = - [y log k + (1-y)log(1-k)]
# with y=1 we obtain b_cross_ent =  - log D(G(z))
# that is what we looking for
gan.compile(loss="binary_crossentropy", optimizer=optimizer)

In [13]:
gan.summary()


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
model_2 (Functional)         (None, 28, 28, 1)         1021752   
_________________________________________________________________
model_3 (Functional)         (None, 1)                 233985    
Total params: 1,255,737
Trainable params: 1,252,713
Non-trainable params: 3,024
_________________________________________________________________


# Few utility functions

In [14]:

# we need to know the dimension of gan input
# gaussian noise
def generate_random_z(gan, num):
    G =  gan.layers[1]
    z_dim = G.layers[0].input.shape[1]
    return np.random.normal(0, 1, size=[num, z_dim])

# immagini reali
def get_real_imgs(ds, num):
    ndx = np.random.randint(0, len(ds), size=num)
    return ds[ndx]

# generami delle immagini
def get_gen_imgs(gan, num):
    noise = generate_random_z(gan=gan, num=num)
    G = gan.layers[1]
    return G.predict(noise)

def show_img_samples(gan, step_no):
    D = gan.layers[2]
    imgs = get_gen_imgs(gan, 25)
    pred_fake = D.predict(imgs) < 0.5

    fig, axes = plt.subplots(5, 5, figsize=(20,20))
    cmaps = { True: "hot", False: "gray" }

    for i in range(5):
        for j in range(5):
            imgno = i*5 + j
            ax = axes[i,j]
            ax.imshow(imgs[imgno], cmap=cmaps[pred_fake[imgno][0]])

    plt.savefig("images/img_{}.png".format(step_no))

In [15]:
z = G.layers[0].input.shape[1]
print(z)

100


# Main train loop

In [16]:
def D_train(ds, gan, batch_size):
    D = gan.layers[2]
    D.trainable = True
    
    # we are gonna use one of the tips & tricks as seen in the theory part
    for i in range(5):
        x_real = get_real_imgs(ds, batch_size) 
        x_gen = get_gen_imgs(gan, batch_size)

        y_real = np.ones([batch_size])
        y_gen = np.zeros([batch_size])

        x = np.concatenate([x_real, x_gen])
        y = np.concatenate([y_real, y_gen])

        perfs = D.train_on_batch(x,y)

    return perfs

def G_train(ds, gan, batch_size):
    D = gan.layers[2]
    D.trainable = False
    
    # *2 because of loss function 1/2
    zs = generate_random_z(gan, batch_size*2)
    y = np.ones([batch_size*2])

    return gan.train_on_batch(zs, y)

def train(gan, ds, steps, batch_size=batch_size):
    D = gan.layers[2]

    for i in range(steps):

        d_loss, d_acc, d_tp, d_tn = D_train(ds, gan, batch_size)
        g_loss = G_train(ds, gan, batch_size)

        if i % 10 == 0:
            print("\ri:{} G[l:{:0.3f}] D[l:{:0.3f} a:{:0.3f} +:{:0.3f} -:{:0.3f}]"
                .format(i,    g_loss,      d_loss,    d_acc,     d_tp,     d_tn), end="")

        if i % 100 == 0:
            show_img_samples(gan, i)

    show_img_samples(gan, steps)

In [17]:
train(gan, X_train, 10000)

i:9590 G[l:0.644] D[l:0.637 a:0.598 +:114.000 -:39.000]